<a href="https://colab.research.google.com/github/sazio/MultiAgentCovid/blob/master/Multiplex_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data and match Agents with Buildings

In [12]:
import pandas as pd 
import glob 

In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [20]:
path = "drive/MyDrive/MAS/Data/Output_GAMA/Data"

agents_path = path + "/Agents"
buildings_path = path + "/Buildings"

agents_files = glob.glob(agents_path + "/*.csv")
buildings_file = glob.glob(buildings_path + "/*.csv")

['drive/MyDrive/MAS/Data/Output_GAMA/Data/Buildings/Buildings.csv']